In [ ]:
# Importing libraries

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import itertools as it
import matplotlib.pyplot as plt

from datetime import datetime
from scipy.stats import multivariate_normal
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

plt.rc('font', size=16)

import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers

# Setting random seed for reproducibility

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
tfp.random.sanitize_seed(seed)

In [ ]:
# Opening the dataframe
ABB_data = pd.read_csv('../input/electricitydemand/Multivariate_Labeled_2 (1).csv', parse_dates=['DateTime'], delimiter=',')
ABB_data.drop(columns="DateTime", inplace=True)
ABB_data

In [ ]:
# Dataframe preprocessing

# Converting data types
ABB_data['CB_First_Floor'] = ABB_data['CB_First_Floor'].astype(np.float32)
ABB_data['CB_Second_Floor'] = ABB_data['CB_Second_Floor'].astype(np.float32)
ABB_data['AvgT'] = ABB_data['AvgT'].astype(np.float32)
ABB_data['AvgH'] = ABB_data['AvgH'].astype(np.float32)
ABB_data['Anomaly_Label'] = ABB_data['Anomaly_Label'].astype(np.float32)
ABB_data['Year'] = ABB_data['Year'].astype(np.float32)
ABB_data['Month'] = ABB_data['Month'].astype(np.float32)
ABB_data['Day'] = ABB_data['Day'].astype(np.float32)
ABB_data['Hour'] = ABB_data['Hour'].astype(np.float32)
ABB_data['Min'] = ABB_data['Min'].astype(np.float32)
ABB_data['Sec'] = ABB_data['Sec'].astype(np.float32)
ABB_data['Day_Of_The_Week'] = ABB_data['Day_Of_The_Week'].astype(np.float32)

# Printing the dataframe's info
ABB_data.info()

In [ ]:
# Function used to plot time series
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(15,10))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
    
# Plotting time series
inspect_dataframe(ABB_data[["CB_First_Floor", "CB_Second_Floor", "Anomaly_Label", "Day_Of_The_Week"]], ABB_data[["CB_First_Floor", "CB_Second_Floor", "Anomaly_Label", "Day_Of_The_Week"]].columns)

In [ ]:
# Columns used for the following split in train, test and validation sets
ABB_data['Test'] = 0
ABB_data['Validation'] = 0
ABB_data['Train_P1'] = 0
ABB_data['Train_P2'] = 0

In [ ]:
# TEST SET

# Some random weeks
week = [89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111]
for j in week:
    Slice_Multivariate_data = ABB_data.iloc[(j)*672:(j+1)*672]
    for k in range((j)*672,((j+1)*672)):
        ABB_data.at[k, 'Test'] = 1
        
j=112
for k in range((j)*672,((j+1)*672)-96):
        ABB_data.at[k, 'Test'] = 1

In [ ]:
# VALIDATION SET

# Some random weeks
week = [55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78]
for j in week:
    Slice_Multivariate_data = ABB_data.iloc[(j)*672:(j+1)*672]
    for k in range((j)*672,(j+1)*672):
        ABB_data.at[k, 'Validation'] = 1

In [ ]:
# TRAINING SET

week = [x for x in np.arange(0, 55, 1)]
for j in week:
    Slice_Univariate_data = ABB_data.iloc[(j)*672:(j+1)*672]
    for k in range((j)*672,(j+1)*672):
        ABB_data.at[k, 'Train_P1'] = 1
    
week = [x for x in np.arange(79, 89, 1)]
for j in week:
    Slice_Univariate_data = ABB_data.iloc[(j)*672:(j+1)*672]
    for k in range((j)*672,(j+1)*672):
        ABB_data.at[k, 'Train_P2'] = 1

In [ ]:
# Plotting time series
inspect_dataframe(ABB_data[["CB_First_Floor", "CB_Second_Floor", "Anomaly_Label", "Day_Of_The_Week", "Train_P1", "Train_P2", "Validation", "Test"]], ABB_data[["CB_First_Floor", "CB_Second_Floor", "Anomaly_Label", "Day_Of_The_Week", "Train_P1", "Train_P2", "Validation", "Test"]].columns)

In [ ]:
# TRAIN, TEST, VALIDATION SETS
X_train_temp_P1 = ABB_data[ABB_data['Train_P1']==1].copy()
X_train_temp_P2 = ABB_data[ABB_data['Train_P2']==1].copy()
X_validation_temp = ABB_data[ABB_data['Validation']==1].copy()
X_test_temp = ABB_data[ABB_data['Test']==1].copy()

In [ ]:
# Training, Validation and Test set - Anomalies (not used for training)
Anomaly_Train_P1 = X_train_temp_P1[['Anomaly_Label']]
Anomaly_Train_P2 = X_train_temp_P2[['Anomaly_Label']]
Anomaly_Validation = X_validation_temp[['Anomaly_Label']]
Anomaly_Test = X_test_temp[['Anomaly_Label']]

In [ ]:
Anomaly_Test.isna().sum()

In [ ]:
X_train_temp_global = pd.concat([X_train_temp_P1,X_train_temp_P2],axis=0)
X_train_temp_global.shape

In [ ]:
# STANDARDIZATION

mean = X_train_temp_global.mean()
std_dev = X_train_temp_global.std()
print(mean, std_dev)

X_train_raw_P1 = (X_train_temp_P1-mean)/std_dev
X_train_raw_P2 = (X_train_temp_P2-mean)/std_dev
X_validation_raw = (X_validation_temp-mean)/std_dev
X_test_raw = (X_test_temp-mean)/std_dev

print(X_test_raw.shape[0]+X_validation_raw.shape[0]+X_train_raw_P1.shape[0]+X_train_raw_P2.shape[0] == ABB_data.shape[0])

In [ ]:
# Anomalies are not considered during the training
X_train_raw_P1 = X_train_raw_P1[['CB_First_Floor', 'CB_Second_Floor', 'Day_Of_The_Week']]
X_train_raw_P2 = X_train_raw_P2[['CB_First_Floor', 'CB_Second_Floor', 'Day_Of_The_Week']]
X_validation_raw = X_validation_raw[['CB_First_Floor', 'CB_Second_Floor', 'Day_Of_The_Week']]
X_test_raw = X_test_raw[['CB_First_Floor', 'CB_Second_Floor', 'Day_Of_The_Week']]
X_train_raw_P1.shape, X_train_raw_P2.shape, X_validation_raw.shape, X_test_raw.shape

In [ ]:
# Plotting time series
inspect_dataframe(X_test_raw, X_test_raw.columns)

## Training Preparation

In [ ]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
tfp.random.sanitize_seed(seed)

In [ ]:
# Training parameters

window = 672       # 1 week
stride = 8         # 1 hour
latent_dim = 10    # Latent dimension
epochs = 500       # Number of epochs (no early stopping)
batch_size = 8     # Batch size
alpha = 0.5
M = 100
noise_factor = [0.2, 0.2, 0.2]

In [ ]:
# Function used to create (overlapped) windows that we will use for the training
def build_sequences(df, anomaly, window=window, stride=stride):
    
    assert len(df) > window
    dataset = []
    anomalies = []
    temp_df = df.copy().values
    temp_an = anomaly.copy().values
    padding_len = (len(df)-window)%stride # Padding computed considering the stride
    print("PADDING: " + str(padding_len)) #DEBUG

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_an.shape[1]), dtype='float32')
        temp_an = np.concatenate((padding,temp_an))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window+1,stride):
        dataset.append(temp_df[idx:idx+window])
        anomalies.append(temp_an[idx:idx+window])

    dataset = np.array(dataset)
    anomalies = np.array(anomalies)
    return dataset, anomalies

In [ ]:
X_train_P1, A_train_P1 = build_sequences(X_train_raw_P1, Anomaly_Train_P1, window=window, stride=stride)
X_train_P2, A_train_P2 = build_sequences(X_train_raw_P2, Anomaly_Train_P2, window=window, stride=stride)
X_val, A_val = build_sequences(X_validation_raw, Anomaly_Validation, window=window, stride=stride)
X_train_P1.shape,X_train_P2.shape

In [ ]:
# Temp test set - plotting purposes
X_test, A_test = build_sequences(X_test_raw, Anomaly_Test, window=window, stride=stride)

In [ ]:
X_train = np.concatenate([X_train_P1,X_train_P2],axis=0)
A_train = np.concatenate([A_train_P1,A_train_P2],axis=0)
X_train.shape, A_train.shape

## Building the model

In [ ]:
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Reparametrization trick
def sample_z1(args):
    z_mean, z_log_var = args
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1]))
    return z_mean + tf.exp(alpha * z_log_var) * eps

# Reparametrization trick
def sample_z2(args):
    z_mean, z_log_var = args
    eps = tf.keras.backend.random_normal(shape=(K.shape(z_mean)[0], K.int_shape(z_mean)[1], K.int_shape(z_mean)[2]))
    return z_mean + tf.exp(alpha * z_log_var) * eps

In [ ]:
class AddNoise(tfk.layers.Layer):
    
    def call(self, inputs):
        data, noise_factor = inputs
        noise = tf.map_fn(fn=lambda t: tf.random.normal((672,3), 0, 1)*noise_factor, elems=data)
        noise_input = data + noise
        
        return noise_input

In [ ]:
class SimScore(tfk.layers.Layer):

    def call(self, inputs):
        seq, h_dim = inputs  # seq: batch x x_dim x h_dim*2

        S = tf.map_fn(fn=lambda t: tf.linalg.matmul(tf.transpose(t), t), elems=seq) # batch x h_dim x h_dim
        S = tf.map_fn(fn=lambda t: t / tf.math.sqrt((tf.cast(h_dim*2, dtype=tf.float32))), elems=S)
        A = tf.map_fn(fn=lambda t: tf.keras.activations.softmax(t), elems=S)
        C = tf.matmul(seq, A)

        return C

In [ ]:
# Building the model

from keras import backend as K
from tensorflow.keras import Input
attention_dim = 10

input_shape = X_train.shape[1:]
output_shape = X_train.shape[1:]

###########
# ENCODER #
###########

encoder_input = tf.keras.Input(shape=input_shape)

noisy_input = AddNoise()([encoder_input, noise_factor])

h_seq, forward_h, forward_c, backward_h, backward_c = tfkl.Bidirectional(tfkl.LSTM(72, activation='tanh', return_sequences=True, return_state=True))(noisy_input)

state_h = tfkl.Concatenate()([forward_h, backward_h])

Cdet = SimScore()([h_seq, window])

c_mean = tfkl.Dense(attention_dim , activation='linear', name="c_mean")(Cdet)
c_log_var = tfkl.Dense(attention_dim , activation='softplus', name="c_var")(Cdet)

# Latent representation: mean + log of std.dev.
z_mean = tfkl.Dense(latent_dim, activation='linear', name="z_mean")(state_h)
z_log_var = tfkl.Dense(latent_dim, activation='softplus', name="z_var")(state_h)

# Sampling a vector from the latent distribution
z = tfkl.Lambda(sample_z1, name='z')([z_mean, z_log_var])
c = tfkl.Lambda(sample_z2, name='c')([c_mean, c_log_var])

encoder = tfk.Model(encoder_input, [z_mean, z_log_var, z, c_mean, c_log_var, c], name='encoder')
print(encoder.summary())

In [ ]:
###########
# DECODER #
###########

z_inputs = Input(shape=(latent_dim, ), name='decoder_input_1')
c_inputs = Input(shape=(window, attention_dim), name='decoder_input_2')

repeated = tfkl.RepeatVector(window)(z_inputs)
concat = tfkl.Concatenate(axis=-1)([repeated, c_inputs])

x = tfkl.Bidirectional(tfkl.LSTM(72, return_sequences=True))(concat)

# OPZIONE 1 - Dense Layers come nel paper
mu = tfkl.Dense(3, activation='linear', name="mu")(x)
log_sigma = tfkl.Dense(3, activation='softplus', name="sigma")(x)

# Define and summarize decoder model
decoder = tfk.Model([z_inputs, c_inputs], [mu, log_sigma], name='decoder')
decoder.summary()

In [ ]:
class VAE(tfk.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tfk.metrics.Mean(name="total_loss")
        self.likelihood_tracker = tfk.metrics.Mean(name="likelihood")
        self.kl_loss_z_tracker = tfk.metrics.Mean(name="kl_loss_z")
        self.kl_loss_c_tracker = tfk.metrics.Mean(name="kl_loss_c")
        self.reconstruction_loss_tracker = tfk.metrics.Mean(name="reconstruction_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.likelihood_tracker,
            self.kl_loss_z_tracker,
            self.kl_loss_c_tracker,
            self.reconstruction_loss_tracker
        ]
    

    def train_step(self, data):
        with tf.GradientTape() as tape:
            
            sample = None
            encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = self.encoder(data)
            decoder_mu, decoder_log_sigma = self.decoder([z,c])
            decoder_sigma = tf.exp(decoder_log_sigma)
            
            pdf_laplace = tfp.distributions.Laplace(decoder_mu, decoder_sigma, validate_args=True, name='Laplace')
                        
            for _ in range(M):
                sample = pdf_laplace.sample() if sample is None else sample + pdf_laplace.sample()
                             
            likelihood = -(pdf_laplace.log_prob(data))
            likelihood = tf.reduce_mean(likelihood, axis=-1)
            likelihood = tf.reduce_mean(likelihood, axis=-1)
                
            decoder_output = sample/M
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, decoder_output), axis=1))
            
            kl_loss_z = -0.5 * (1 + encoder_log_var - tf.square(encoder_mu) - tf.exp(encoder_log_var))
            kl_loss_z = tf.reduce_mean(tf.reduce_sum(kl_loss_z, axis=1))
            
            kl_loss_c = -0.5 * (1 + c_log_var - tf.square(c_mean) - tf.exp(c_log_var))
            kl_loss_c = tf.keras.backend.sum(kl_loss_c, axis=1)
            kl_loss_c = tf.reduce_mean(kl_loss_c, axis=1)

            total_loss = 4*likelihood + 2*(kl_loss_z + 7*kl_loss_c) + reconstruction_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.likelihood_tracker.update_state(likelihood)
        self.kl_loss_z_tracker.update_state(kl_loss_z)
        self.kl_loss_c_tracker.update_state(kl_loss_c)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "likelihood": self.likelihood_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result()
        }
    
    
    
    def test_step(self, data): # https://github.com/keras-team/keras-io/issues/38

        sample = None
        encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = self.encoder(data)
        decoder_mu, decoder_log_sigma = self.decoder([z,c])
        decoder_sigma = tf.exp(decoder_log_sigma)
            
        pdf_laplace = tfp.distributions.Laplace(decoder_mu, decoder_sigma, validate_args=True, name='Laplace')
                        
        for _ in range(M):
            sample = pdf_laplace.sample() if sample is None else sample + pdf_laplace.sample()
                             
        likelihood = -(pdf_laplace.log_prob(data))
        likelihood = tf.reduce_mean(likelihood, axis=-1)
        likelihood = tf.reduce_mean(likelihood, axis=-1)
                
        decoder_output = sample/M
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, decoder_output), axis=1))
            
        kl_loss_z = -0.5 * (1 + encoder_log_var - tf.square(encoder_mu) - tf.exp(encoder_log_var))
        kl_loss_z = tf.reduce_mean(tf.reduce_sum(kl_loss_z, axis=1))
            
        kl_loss_c = -0.5 * (1 + c_log_var - tf.square(c_mean) - tf.exp(c_log_var))
        kl_loss_c = tf.keras.backend.sum(kl_loss_c, axis=1)
        kl_loss_c = tf.reduce_mean(kl_loss_c, axis=1)

        total_loss = 4*likelihood + 2*(kl_loss_z + 7*kl_loss_c) + reconstruction_loss
            
        self.total_loss_tracker.update_state(total_loss)
        self.likelihood_tracker.update_state(likelihood)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_z_tracker.update_state(kl_loss_z)
        self.kl_loss_c_tracker.update_state(kl_loss_c)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "likelihood": self.likelihood_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_c": self.kl_loss_c_tracker.result(),
            "kl_z": self.kl_loss_z_tracker.result(),
        }

In [ ]:
vae = VAE(encoder, decoder)

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

vae.compile(optimizer=tfk.optimizers.Adam())

vae.fit(x = X_train,
        validation_data = (X_val, None),
        epochs=epochs, 
        batch_size=batch_size,
        callbacks=[tfk.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True), tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-5)]
       )

## Example of prediction on the test set

In [ ]:
# Check - Variances on train set
w = X_train[0,:,:]
encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = encoder(np.expand_dims(w, axis=0))
print(tf.exp(c_log_var))
decoder_mu, decoder_log_sigma = decoder([z,c])
print(tf.exp(decoder_log_sigma))

In [ ]:
def inspect_multivariate_prediction(X, y, a, pred, columns, telescope, idx=None):

    figs, axs = plt.subplots(len(columns) + 1, 1, sharex=True, figsize=(17,10))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[0,:,i])
        axs[i].plot(np.arange(len(X[0,:,i]), len(X[0,:,i])+telescope), y[0,:,i], color='orange')
        axs[i].plot(np.arange(len(X[0,:,i]), len(X[0,:,i])+telescope), pred[0,:,i], color='green')
        axs[i].set_title(col)
           
    axs[len(columns)].plot(np.arange(len(X[0,:,i])), a[:,0])
    axs[len(columns)].plot(np.arange(len(X[0,:,i]), len(X[0,:,i])+telescope), a[:,0])
    axs[len(columns)].set_title('Anomaly')
    plt.show()

In [ ]:
# Example of prediction on the test set

x = 0

w = X_test[x,:,:] 
w = tf.expand_dims(w, 0)

sample = None
encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = encoder(w)
decoder_mu, decoder_log_sigma = decoder([z,c])
decoder_sigma = tf.exp(decoder_log_sigma)

a = A_test[x,:]         
pdf_laplace = tfp.distributions.Laplace(decoder_mu, decoder_sigma, validate_args=True, name='Laplace')

for _ in range(M):
    sample = pdf_laplace.sample() if sample is None else sample + pdf_laplace.sample()
reconstruction = sample/M

# Plotting predictions on the test set
target_labels = X_test_raw.columns
inspect_multivariate_prediction(w, w, a, reconstruction, target_labels, window)

In [ ]:
# Example of prediction on the test set

x = 84

w = X_test[x,:,:] 
w = tf.expand_dims(w, 0)

sample = None
encoder_mu, encoder_log_var, z, c_mean, c_log_var, c = encoder(w)
decoder_mu, decoder_log_sigma = decoder([z,c])
decoder_sigma = tf.exp(decoder_log_sigma)

a = A_test[x,:]         
pdf_laplace = tfp.distributions.Laplace(decoder_mu, decoder_sigma, validate_args=True, name='Laplace')

for _ in range(M):
    sample = pdf_laplace.sample() if sample is None else sample + pdf_laplace.sample()
reconstruction = sample/M

# Plotting predictions on the test set
target_labels = X_test_raw.columns
inspect_multivariate_prediction(w, w, a, reconstruction, target_labels, window)

## Computing reconstruction probability

In [ ]:
# Function used to create non-overlapped windows starting from the TRAINING set
def build_sequences_reconstruction(df, window, stride):
    dataset = []
    temp_df = df.copy().values
    padding_len = (len(df)-window)%stride

    for idx in np.arange(0,len(temp_df)-window+1,stride):
        dataset.append(temp_df[idx:idx+window])

    dataset = np.array(dataset)
    return dataset

In [ ]:
X_test_overlapped = build_sequences_reconstruction(X_test_raw, window=window, stride=stride)
X_test_overlapped.shape

In [ ]:
reconstructed_prob_1 = np.zeros(X_test_raw.shape[0], dtype='float32') # Array that will contain rec prob
reconstructed_prob_2 = np.zeros(X_test_raw.shape[0], dtype='float32') # Array that will contain rec prob
reconstructed_prob_written_1 = np.zeros(X_test_raw.shape[0], dtype='float32') # Array that will contain rec prob
reconstructed_prob_written_2 = np.zeros(X_test_raw.shape[0], dtype='float32') # Array that will contain rec prob
M=25

for j in range (X_test_overlapped.shape[0]): # Consider all the windows in X_test_overlapped one at a time ...
    
    if(j%100==0): print(j)
    
    w = X_test_overlapped[j,:,:] 
    w = tf.expand_dims(w, 0)
    
    z_mean, z_log_var, z, c_mean, c_log_var, c = encoder(w) # Sampling only 1 time in the latent space
    
    if (j == 0):
        
        for m in range(M):
            
            decoder_mu, decoder_log_sigma = decoder([z,c])
            decoder_sigma = tf.exp(decoder_log_sigma)
            
            for i in range(window):  
                pdf_laplace_1 = tfp.distributions.Laplace(decoder_mu[0, i, 0], decoder_sigma[0, i, 0], validate_args=True, name='Laplace1')
                pdf_laplace_2 = tfp.distributions.Laplace(decoder_mu[0, i, 1], decoder_sigma[0, i, 1], validate_args=True, name='Laplace2')
                p_l_1 = -(pdf_laplace_1.log_prob(X_test_overlapped[j, i, 0])) 
                p_l_2 = -(pdf_laplace_2.log_prob(X_test_overlapped[j, i, 1])) 
                reconstructed_prob_1[i] = reconstructed_prob_1[i] + p_l_1
                reconstructed_prob_2[i] = reconstructed_prob_2[i] + p_l_2
                reconstructed_prob_written_1[i] = reconstructed_prob_written_1[i] + 1
                reconstructed_prob_written_2[i] = reconstructed_prob_written_2[i] + 1
                
    else:
        
        for m in range(M):
            
            decoder_mu, decoder_log_sigma = decoder([z,c])
            decoder_sigma = tf.exp(decoder_log_sigma)
            
            for i in range(stride):
                pdf_laplace_1 = tfp.distributions.Laplace(decoder_mu[0, window-stride+i, 0], decoder_sigma[0, window-stride+i, 0], validate_args=True)
                pdf_laplace_2 = tfp.distributions.Laplace(decoder_mu[0, window-stride+i, 1], decoder_sigma[0, window-stride+i, 1], validate_args=True)
                p_l_1 = -(pdf_laplace_1.log_prob(X_test_overlapped[j, window-stride+i, 0]))   
                p_l_2 = -(pdf_laplace_2.log_prob(X_test_overlapped[j, window-stride+i, 1])) 
                if window + (j-1)*stride + i < X_test_raw.shape[0]: # To avoid problems related to padding
                    reconstructed_prob_1[window + (j-1) * stride + i] = reconstructed_prob_1[window + (j-1) * stride + i] + p_l_1
                    reconstructed_prob_2[window + (j-1) * stride + i] = reconstructed_prob_2[window + (j-1) * stride + i] + p_l_2
                    reconstructed_prob_written_1[window + (j-1) * stride + i] = reconstructed_prob_written_1[window + (j-1) * stride + i] + 1
                    reconstructed_prob_written_2[window + (j-1) * stride + i] = reconstructed_prob_written_2[window + (j-1) * stride + i] + 1
                    
        
reconstructed_prob_1 /= M
reconstructed_prob_2 /= M
reconstructed_prob_1.shape, reconstructed_prob_2.shape, reconstructed_prob_1.max(), reconstructed_prob_1.min(), reconstructed_prob_2.max(), reconstructed_prob_2.min()

In [ ]:
reconstructed_prob_written_1.sum(), reconstructed_prob_written_2.sum(), X_test_raw.shape[0]

In [ ]:
# Checking if we've written correct things inside "reconstruction_prob"
check = True
for i in range(reconstructed_prob_written_1.shape[0]):
    if reconstructed_prob_written_1[i] != M: # Each cell written M times
        check = False
        
print(check) # If true, everything okay

In [ ]:
# Counting the number of elements in "reconstructed_prob" that are equal to 0
counter = 0
for i in range(reconstructed_prob_1.shape[0]):
    if reconstructed_prob_1[i] == 0:
        counter += 1
        
print(counter)

In [ ]:
reconstructed_prob = reconstructed_prob_1 + reconstructed_prob_2

In [ ]:
# Function used to predict anomalies
def predict_anomalies(reconstruction_probabilities, threshold):
    
    # Inizially we don't have predicted anomalies
    anomaly_predicted = np.zeros(shape=X_test_raw.shape[0])
    
    for i in range(reconstruction_probabilities.shape[0]): 
        if reconstruction_probabilities[i] > threshold:
            anomaly_predicted[i] = 1
            
    return anomaly_predicted

In [ ]:
def select_best_threshold(reconstructed_prob, step, anomaly_true):
    
    min_prob = reconstructed_prob.min()
    max_prob = reconstructed_prob.max()
    max_F1 = 0
    best_threshold = 0
    counter = 0
    
    print("best threshold = " + str(best_threshold) + ", max F1 = " + str(max_F1))
    
    thresholds = [x for x in np.arange(min_prob, max_prob, step)]
    
    for threshold in thresholds:
        
        # print("Trying threshold: " + str(threshold))
        
        if(counter<1000):
            
            anomaly_predicted = predict_anomalies(reconstructed_prob, threshold)
            f1 = f1_score(y_true=anomaly_true, y_pred=anomaly_predicted)
            if f1 > max_F1:
                max_F1 = f1
                best_threshold = threshold
                print("best threshold =" + str(best_threshold) + ", max F1 = " + str(max_F1))
            else:
                counter = counter+1
        
        else:
            return best_threshold

In [ ]:
step = 1/1000
threshold = select_best_threshold(reconstructed_prob, step, Anomaly_Test)
anomaly_predicted = predict_anomalies(reconstructed_prob, threshold)

threshold, anomaly_predicted.sum(), anomaly_predicted.shape

In [ ]:
Anomaly_Test.reset_index(inplace=True)
Anomaly_Test.drop(columns='index', inplace=True)
Anomaly_Test

In [ ]:
X_test_raw.reset_index(inplace=True)
X_test_raw.drop(columns='index', inplace=True)
X_test_raw

In [ ]:
Anomaly_Test.sum()

In [ ]:
X_test_raw['Anomaly_Label_GT'] = Anomaly_Test # Ground truth anomalies
X_test_raw['Anomaly_Predicted'] = anomaly_predicted 
X_test_raw['Reconstruction_Probability'] = reconstructed_prob
X_test_raw.head()

In [ ]:
# Function used to plot time series
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(30,20))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()

In [ ]:
def inspect_multivariate_prediction(X, y, a_true, a_predicted, reconstruction_prob, columns): 
   
    figs, axs = plt.subplots(5, 1, sharex=True, figsize=(17,17)) 
    for i, col in enumerate(columns[:2]): 
        axs[i].plot(np.arange(len(X[:,i])), X[:,i]) 
        axs[i].plot(np.arange(len(X[:,i]), len(X[:,i])*2), y[:,i], color='orange')  
        axs[i].set_title(col) 
                 
    axs[2].plot(np.arange(len(X[:,i])), a_true) 
    axs[2].plot(np.arange(len(X[:,i]), len(X[:,i])*2), a_true) 
    axs[2].set_title('Anomaly_Label_GT') 
    
    axs[3].plot(np.arange(len(X[:,i])), a_predicted) 
    axs[3].plot(np.arange(len(X[:,i]), len(X[:,i])*2), a_predicted) 
    axs[3].set_title('Anomaly_Predicted')
    
    axs[4].plot(np.arange(len(X[:,i])), reconstruction_prob) 
    axs[4].plot(np.arange(len(X[:,i]), len(X[:,i])*2), reconstruction_prob) 
    axs[4].set_title('Reconstrution_Probability') 
    plt.show()

In [ ]:
import datetime as dt

i=4
j=i+3

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=4
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=5
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=6
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=7
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=8
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=9
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
import datetime as dt

i=10
j=i+1

X = np.array(X_test_raw.iloc[i*672:j*672][['CB_First_Floor', 'CB_Second_Floor']])
anomaly_true = X_test_raw.iloc[i*672:j*672]['Anomaly_Label_GT'].copy()
anomaly_prediction = X_test_raw.iloc[i*672:j*672]['Anomaly_Predicted'].copy()
reconstruction_probability = X_test_raw.iloc[i*672:j*672]['Reconstruction_Probability'].copy()

# Plotting predictionsreconstruction_loss2 = tf.reduce_mean(tf.reduce_sum(tfk.losses.mse(data, y), axis=1))
inspect_multivariate_prediction(X, X, anomaly_true, anomaly_prediction, reconstruction_probability, target_labels)

In [ ]:
anomaly_predicted = anomaly_predicted

# Compute the confusion matrix
cm = confusion_matrix(y_true=Anomaly_Test, y_pred=anomaly_predicted)

# Compute the classification metrics
accuracy = accuracy_score(y_true=Anomaly_Test, y_pred=anomaly_predicted)
precision = precision_score(y_true=Anomaly_Test, y_pred=anomaly_predicted)
recall = recall_score(y_true=Anomaly_Test, y_pred=anomaly_predicted)
f1 = f1_score(y_true=Anomaly_Test, y_pred=anomaly_predicted)
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(5, 5))
ax.matshow(cm, cmap=plt.cm.Oranges, alpha=0.3)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(x=j, y=i,s=cm[i, j], va='center', ha='center', size='xx-large')
        
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()